In [30]:
from langchain.llms import CTransformers
from langchain import PromptTemplate, LLMChain
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [31]:
llm = CTransformers(model="TheBloke/Llama-2-7B-Chat-GGML", model_file = 'llama-2-7b-chat.ggmlv3.q2_K.bin', callbacks=[StreamingStdOutCallbackHandler()])

Fetching 1 files: 100%|██████████| 1/1 [00:00<00:00, 9731.56it/s]


In [32]:
list_genres = [
    'acoustic',
    'alt-rock',
    'alternative',
    'ambient',
    'blues',
    'classical',
    'country',
    'dance',
    'disco',
    'edm',
    'electronic',
    'folk',
    'funk',
    'hip-hop',
    'house',
    'indie',
    'jazz',
    'k-pop',
    'latin',
    'metal',
    'pop',
    'punk',
    'r-n-b',
    'reggae',
    'rock',
    'soul',
    'techno',
    'trance',
    'reggaeton',
    'singer-songwriter'
]

In [33]:
template = """
[INST] <<SYS>>
I give you a sentence, answer in a very specific way.
The objective is : based on a sentence describing a mood, I need get a list of characteristics allowing me to generate a playlist accordingly.
I want you to format the answer this exact way:
"target_acousticness: x,
target_danceability: x,
target_energy: x,
target_instrumentalness: x,
target_speechiness: x,
genre: x"
For parameters that start with "target_", replace the "x" with a value between 0.0 and 1.0 that you consider the most appropriate based on my sentence
For the "genre" parameter, replace the "x" with the music genre you think is best suited from the following list : {list_genres}.
So, for each sentence I will send, I want you to respond ONLY in the above format. NO OTHER TEXT, NO INTRO OR OUTRO MESSAGE.
<</SYS>>
{text}[/INST]
"""

prompt = PromptTemplate(template=template, input_variables=["list_genres","text"])

In [34]:
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [35]:
response = llm_chain.run(list_genres=list_genres, text="I'm feeling kinda depressed today and i'd like to listen to some lofi i think")

target_acousticness: 0.5,
target_danceability: 0.3,
target_energy: 0.4,
target_instrumentalness: 0.7,
target_speechiness: 0.6,
genre: alternative

------------------

In [ ]:
response

In [ ]:
normalized_string = response.replace(',', '').replace(' ', '').replace('\n', '').lower()

In [ ]:
target_acousticness = float(re.search(r'target_acousticness:([\d.]+)', normalized_string).group(1))

In [ ]:
#target_acousticness = float(re.search(r'target_acousticness:([\d.]+)', normalized_string).group(1))
#target_danceability = float(re.search(r'target_danceability:([\d.]+)', normalized_string).group(1))
#target_energy = float(re.search(r'target_energy:([\d.]+)', normalized_string).group(1))
#target_instrumentalness = float(re.search(r'target_instrumentalness:([\d.]+)', normalized_string).group(1))
#target_speechiness = float(re.search(r'target_speechiness:([\d.]+)', normalized_string).group(1))


# Parsing response

In [ ]:
import re

def parse_mood_string(mood_string):
    # Normalize the string: remove commas, spaces, and convert to lowercase
    normalized_string = mood_string.replace(',', '').replace(' ', '').replace('\n', '').lower()

    # Extract values using regular expressions
    target_acousticness = float(re.search(r'target_acousticness:([\d.]+)', normalized_string).group(1))
    target_danceability = float(re.search(r'target_danceability:([\d.]+)', normalized_string).group(1))
    target_energy = float(re.search(r'target_energy:([\d.]+)', normalized_string).group(1))
    target_instrumentalness = float(re.search(r'target_instrumentalness:([\d.]+)', normalized_string).group(1))
    target_speechiness = float(re.search(r'target_speechiness:([\d.]+)', normalized_string).group(1))
    genre = re.search(r"genre:('[a-z]+')", normalized_string).group(1).replace("'", "")

    # Create the dictionary
    mood_data = {
        "target_acousticness": target_acousticness,
        "target_danceability": target_danceability,
        "target_energy": target_energy,
        "target_instrumentalness": target_instrumentalness,
        "target_speechiness": target_speechiness,
        "genre": genre
    }

    return mood_data

In [ ]:
print(parse_mood_string(response))